![A soccer pitch for an international match.](soccer-pitch.jpg)

You're working as a sports journalist at a major online sports media company, specializing in soccer analysis and reporting. You've been watching both men's and women's international soccer matches for a number of years, and your gut instinct tells you that more goals are scored in women's international football matches than men's. This would make an interesting investigative article that your subscribers are bound to love, but you'll need to perform a valid statistical hypothesis test to be sure!

While scoping this project, you acknowledge that the sport has changed a lot over the years, and performances likely vary a lot depending on the tournament, so you decide to limit the data used in the analysis to only official `FIFA World Cup` matches (not including qualifiers) since `2002-01-01`.

You create two datasets containing the results of every official men's and women's international football match since the 19th century, which you scraped from a reliable online source. This data is stored in two CSV files: `women_results.csv` and `men_results.csv`.

The question you are trying to determine the answer to is:

> Are more goals scored in women's international soccer matches than men's?

You assume a **10% significance level**, and use the following null and alternative hypotheses:

$H_0$ : The mean number of goals scored in women's international soccer matches is the same as men's.

$H_A$ : The mean number of goals scored in women's international soccer matches is greater than men's.

In [70]:
# Import the libraries
import pandas as pd
import numpy as np
from scipy.stats import t, shapiro, mannwhitneyu
import pingouin

In [71]:
# Read the men's files
men = pd.read_csv("men_results.csv", parse_dates = ["date"])
men.head()

,Unnamed: 0,date,home_team,away_team,home_score,away_score,tournament
0,0,1872-11-30,Scotland,England,0,0,Friendly
1,1,1873-03-08,England,Scotland,4,2,Friendly
2,2,1874-03-07,Scotland,England,2,1,Friendly
3,3,1875-03-06,England,Scotland,2,2,Friendly
4,4,1876-03-04,Scotland,England,3,0,Friendly


In [72]:
# Check the information of men dataframe
men.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44353 entries, 0 to 44352
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  44353 non-null  int64         
 1   date        44353 non-null  datetime64[ns]
 2   home_team   44353 non-null  object        
 3   away_team   44353 non-null  object        
 4   home_score  44353 non-null  int64         
 5   away_score  44353 non-null  int64         
 6   tournament  44353 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 2.4+ MB


In [73]:
# Create the column for the total goals per match
men["total_score"] = men["home_score"] + men["away_score"]
# Drop unused columns
men = men.drop(["Unnamed: 0", "home_score", "away_score"], axis = 1)
# Select only FIFA World Cup matches since 2002-01-01
men_df = men[(men["tournament"] == "FIFA World Cup")&(men["date"] >= "2002-01-01")].reset_index(drop = True)
men_df.head()

,date,home_team,away_team,tournament,total_score
0,2002-05-31,France,Senegal,FIFA World Cup,1
1,2002-06-01,Germany,Saudi Arabia,FIFA World Cup,8
2,2002-06-01,Republic of Ireland,Cameroon,FIFA World Cup,2
3,2002-06-01,Uruguay,Denmark,FIFA World Cup,3
4,2002-06-02,Argentina,Nigeria,FIFA World Cup,1


In [74]:
# Read the women's file
women = pd.read_csv("women_results.csv", parse_dates = ["date"])
women.head()

,Unnamed: 0,date,home_team,away_team,home_score,away_score,tournament
0,0,1969-11-01,Italy,France,1,0,Euro
1,1,1969-11-01,Denmark,England,4,3,Euro
2,2,1969-11-02,England,France,2,0,Euro
3,3,1969-11-02,Italy,Denmark,3,1,Euro
4,4,1975-08-25,Thailand,Australia,3,2,AFC Championship


In [75]:
# Check the information of women dataframe
women.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884 entries, 0 to 4883
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  4884 non-null   int64         
 1   date        4884 non-null   datetime64[ns]
 2   home_team   4884 non-null   object        
 3   away_team   4884 non-null   object        
 4   home_score  4884 non-null   int64         
 5   away_score  4884 non-null   int64         
 6   tournament  4884 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 267.2+ KB


In [76]:
# Create the column for the total goals per match
women["total_score"] = women["home_score"] + women["away_score"]
# Drop unused columns
women = women.drop(["Unnamed: 0", "home_score", "away_score"], axis = 1)
# Select only FIFA World Cup matches since 2002-01-01
women_df = women[(women["tournament"] == "FIFA World Cup")&(women["date"] >= "2002-01-01")]\
.reset_index(drop = True)
women_df.head()

,date,home_team,away_team,tournament,total_score
0,2003-09-20,Nigeria,North Korea,FIFA World Cup,3
1,2003-09-20,Norway,France,FIFA World Cup,2
2,2003-09-20,Germany,Canada,FIFA World Cup,5
3,2003-09-20,Japan,Argentina,FIFA World Cup,6
4,2003-09-21,United States,Sweden,FIFA World Cup,4


# We will use either t-test or Wilcoxon-Mann-Whitney test if assumptions for t-test are not met.

In [77]:
# Verify whether sample size in each groups is enough to run t-test
print(len(men_df) > 30)
print(len(women_df) > 30)

True
True


In [78]:
# Test for equal variances of the two sample groups
men_var = men_df["total_score"].var()
wom_var = women_df["total_score"].var()
print(wom_var/men_var)

1.4976928798069002


In [79]:
# Check whether sample of each group have normal distribution
st_men, pval_men = shapiro(men_df["total_score"])
st_wom, pval_wom = shapiro(women["total_score"])
print("p_value for men data is {} and for women data is {}. Data are not normally distributed, therefore, we will use Wilcoxon-Mann-Whitney test in this case.".format(pval_men, pval_wom))

p_value for men data is 8.894154401688226e-13 and for women data is 0.0. Data are not normally distributed, therefore, we will use Wilcoxon-Mann-Whitney test in this case.


In [80]:
# Run the test 
mwu = mannwhitneyu(x = women_df["total_score"], y = men_df["total_score"], alternative = "greater")
# Get the p_value
pval = mwu.pvalue

In [81]:
# Determine the hypothesis results 
if pval <= 0.1:
    result = "reject"
else:
    result = "fail to reject"

In [82]:
# Store the results in dictionary
result_dict = {"p_val": pval, "result": result}
result_dict

{'p_val': 0.005106609825443641, 'result': 'reject'}